# NLP Assignment 1

In this assignment we are meant to use the corpus (plain text) to generate N-gram models where n: 2-6. <br>
The user should enter number of words (tokens) in the desired sentence, for example 10 words and should enter one word. <br>
The output should have 10 words/tokens. We also going to evaluate 8 test samples by human expert (myself or my friend).

### The Code

We start by importing the neccessary libraries:

In [1]:
import os
import nltk
from nltk.corpus import stopwords
import re
import string
from collections import defaultdict
from random import choice

Setting the corpus path

In [2]:
root_path = 'Corpus'

Here the user is allowed to set the desired n-gram.
For testing purposes we sat it to 3 

In [3]:
n = int(input("Enter the value of n for the n-gram model: "))
print('n value:', n)

n value: 3


Here we declare 4 dictionaries that are used to store n-grams, n-grams frequency, n-gram probability

In [4]:
n_gram_counts = defaultdict(int)
n_minus1_gram_counts = defaultdict(int)
n_gram_frequency = defaultdict(int)
n_gram_probs = defaultdict(dict)

read_file function reads the passed file and return it.

In [5]:
def read_file(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        text = file.read()
        return text

preprocess_text as the name says it preprocess the text, toknize it and remove the non-arabic characters and the punctuation

In [6]:
def preprocess_text(text):
    text = re.sub(r'[^\u0600-\u06FF\s]+', '', text) # Remove all non-Arabic characters
    words = nltk.word_tokenize(text) # Tokenize the words
    
    filtered_words = [word for word in words if word not in string.punctuation and word != "،"] # Remove punctuation
    
    # arabic_stop_words = set(stopwords.words('arabic')) # Here we get the stop words of arabic مثل: هذه, هذا, و
    # filtered_words = [word for word in words if word.casefold() not in arabic_stop_words and word.isalpha()] # Here we remove these stop words and numbers
    # filtered_words = [word for word in filtered_words if word not in string.punctuation] # Remove punctuation

    return filtered_words

here we loop through the files in the corpus and apply the prevois two functions then store the results in n_gram_counts
(💡Note: The whole corpus are being used.)

In [7]:
for dir, _, files in os.walk(root_path):
    for file in files:
        if file.endswith('.txt'):
            file_path = os.path.join(dir, file)
            text = read_file(file_path)
            words = preprocess_text(text)
            for i in range(len(words) - n + 1):
                n_gram = tuple(words[i:i+n])
                n_gram_counts[n_gram] += 1

This piece of code maps a n-1 gram (i.e. the first n-1 words in a n-gram) to the total count of all n-grams that start with that same n-1 gram.

In [8]:
for n_gram, count in n_gram_counts.items():
    n_minus1_gram = n_gram[:-1]
    n_minus1_gram_counts[n_minus1_gram] += count

Here we get the top 10 most frequent n-grams

In [9]:
top_n = 10
sorted_n_grams = sorted(n_gram_counts.items(), key=lambda x: x[1], reverse=True)[:top_n]

print(f"Top {top_n} most frequent {n}-grams:")
for n_gram, count in sorted_n_grams:
    print(f"{n_gram}: {count}")

Top 10 most frequent 3-grams:
('صلى', 'الله', 'عليه'): 186980
('الله', 'عليه', 'وسلم'): 162840
('الله', 'صلى', 'الله'): 98126
('رسول', 'الله', 'صلى'): 93524
('عبد', 'الله', 'بن'): 73723
('النبي', 'صلى', 'الله'): 68501
('بن', 'عبد', 'الله'): 30801
('عبد', 'الرحمن', 'بن'): 21843
('رضي', 'الله', 'عنه'): 21212
('الله', 'عز', 'وجل'): 20388


This for loop calculate the probabilities of the n-grams

In [10]:
for n_gram, count in n_gram_counts.items():
    n_minus1_gram = n_gram[:-1]
    prob = count / n_minus1_gram_counts[n_minus1_gram]
    n_gram_probs[n_minus1_gram][n_gram[-1]] = prob

Here we print the probabilties, 10 samples of it.

In [11]:
temp_counter = 0
for n_minus1_gram, prob_dict in n_gram_probs.items():
    print(f"Top 10 probabilities for {n_minus1_gram}:")
    sorted_probs = sorted(prob_dict.items(), key=lambda x: x[1], reverse=True)[:10]
    for next_word, prob in sorted_probs:
        print(f"{next_word}: {prob}")
    print()
    temp_counter += 1
    if temp_counter > 10:
        break

Top 10 probabilities for ('بسم', 'الله'):
الرحمن: 0.8827483196415236
مجراها: 0.012447099825740602
والله: 0.006472491909385114
قال: 0.004729897933781429
وعلى: 0.003236245954692557
الذي: 0.0027383619616629324
أرقيك: 0.0027383619616629324
خير: 0.0024894199651481204
ثم: 0.0022404779686333084
فلما: 0.0019915359721184964

Top 10 probabilities for ('الله', 'الرحمن'):
الرحيم: 0.8947368421052632
الرحيم،: 0.09844559585492228
الرحيم؟: 0.00218161985274066
الرحيم؛: 0.000545404963185165
على: 0.0002727024815925825
عز: 0.0002727024815925825
ص: 0.0002727024815925825
حتى: 0.0002727024815925825
فجرت: 0.0002727024815925825
فلما: 0.0002727024815925825

Top 10 probabilities for ('الرحمن', 'الرحيم'):
قوله: 0.08283175695041559
سورة: 0.052163943823445115
من: 0.049011177987962166
قال: 0.039839495557466326
الحمد: 0.03210088850673545
كتاب: 0.02722843221553454
هذا: 0.025508741759816565
رب: 0.020349670392662653
نا: 0.01948982516480367
القول: 0.01834336486099169

Top 10 probabilities for ('الرحيم', 'الحمد'):
لله: 0.

This method predicts the next word

In [12]:
def predict_next_n_words(initial_words, num_words):
    words = preprocess_text(initial_words)
    num_words += n - 1
    while len(words) < num_words:
        n_minus1_gram = tuple(words[-(n-1):])
        if n_minus1_gram not in n_gram_probs:
            break
        next_word = max(n_gram_probs[n_minus1_gram], key=n_gram_probs[n_minus1_gram].get)
        words.append(next_word)
    return ' '.join(words)

This while loop prompt for the user the required number of initial words + how many words he want.
An example is showing in the output.

In [ ]:
while True:
    initial_words = input(f"Enter {n-1} initial words (separated by spaces) or 'q' to quit: ")
    if initial_words == 'q':
        break
    num_words = int(input("Enter the number of words you want to generate: "))
    prediction = predict_next_n_words(initial_words, num_words)
    print(f"Generated text: {prediction}")




Generated text: السلام عليكم ورحمة الله وبركاته فقال له يا أبا عبد الله بن<br>
Generated text: محمد صلى الله عليه وسلم قال من كان في ذلك قال أهل<br>
Generated text: محمد صلى الله عليه وسلم قال من كان في ذلك قال أهل<br>
Generated text: ابو بكر بن أبي طالب رضي الله عنه قال قال رسول الله<br>
Generated text: علي بن أبي طالب رضي الله عنه قال قال رسول الله صلى الله عليه وسلم قال من كان في ذلك قال أهل<br>
Generated text: يا غلام أعطه أربعة آلاف درهم قال أبو جعفر يعني بذلك جل<br>
Generated text: ابو احمد بن محمد بن عبد الله بن أبي طالب رضي الله<br>
Generated text: ابو جعفر محمد بن عبد الله بن أبي طالب رضي الله عنه<br>
Generated text: عبد الله بن أبي طالب رضي الله عنه قال قال رسول الله<br>
Generated text: عبد الرحمن بن أبي طالب رضي الله عنه قال قال رسول الله<br>
Generated text: مالك يوم الدين قال أبو جعفر يعني بذلك جل ثناؤه بقوله ومن<br>
Generated text: إياك نعبد وإياك نستعين اهدنا الصراط المستقيم صراط الذين أنعمت عليهم غير